### Задание к теме «Гибридные рекомендательные системы»

1. Датасет ml-latest
1. Вспомнить подходы, которые мы разбирали
1. Выбрать понравившийся подход к гибридным системам
1. Написать свою

In [1]:
import pandas as pd
import numpy as np

In [100]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [104]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [105]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [106]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [107]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [108]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [110]:
dataset[dataset.uid==1.0] 

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
325,1.0,Grumpier Old Men (1995),4.0
433,1.0,Heat (1995),4.0
2107,1.0,Seven (a.k.a. Se7en) (1995),5.0
2379,1.0,"Usual Suspects, The (1995)",5.0
...,...,...,...
56820,1.0,Shaft (2000),4.0
57280,1.0,X-Men (2000),5.0
57461,1.0,What About Bob? (1991),4.0
59174,1.0,Transformers: The Movie (1986),4.0


Идея гибридной системы в следующем:
- составляем топ фильмов для просматриваемого пользователя
- находим фильмы, похожие на найденный топ (алгоритм к-neibourghs)
- оцениваем подборку фильмов (задача регрессии)
- в выдачу попадают n фильмов с наиболее высокой оценкой

In [116]:
movie_pivot = dataset.pivot_table( index='iid', columns='uid', values='rating', fill_value=0)
movie_pivot.head()

uid,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
iid,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,4.0
'Hellboy': The Seeds of Creation (2004),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
'Round Midnight (1986),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
'Salem's Lot (2004),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
'Til There Was You (1997),0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0


In [117]:
from scipy.sparse import csr_matrix
movie_sparse=csr_matrix(movie_pivot)

In [118]:
# 1я модель, определяющая схожесть фильмов
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(n_neighbors=20,algorithm='brute',metric='cosine')

In [119]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

Например, для фильма Toy Story (1995) похожими будут выбраны следующие:

In [131]:
film_title = 'Toy Story (1995)'
distances,suggestions=model.kneighbors(movie_pivot.loc[film_title,:].values.reshape(1,-1))

In [133]:
for i in range(len(suggestions[0])):
    if movie_pivot.index[suggestions[0][i]] == film_title:
        continue
    print(movie_pivot.index[suggestions[0][i]])

Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Lion King, The (1994)
Star Wars: Episode VI - Return of the Jedi (1983)
Mission: Impossible (1996)
Groundhog Day (1993)
Back to the Future (1985)
Shrek (2001)
Aladdin (1992)
Apollo 13 (1995)
Pulp Fiction (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)
Willy Wonka & the Chocolate Factory (1971)
Men in Black (a.k.a. MIB) (1997)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Shawshank Redemption, The (1994)


In [140]:
# 2я модель, выставляющая фильму оценку
# В предыдущей домашней работе на тех же данны алгоритм SVD давал наилучший результат из нескольких рассматриваемых,
# поэтому воспользуюсь им
# И параметры возьму те, которые были признаны оптимальными в результате подбора в ходе предыдущей ДЗ

In [136]:
reader = Reader(rating_scale=(dataset.rating.min(), dataset.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [138]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=146)

In [139]:
algo = SVD(n_factors=80, n_epochs=40, lr_all=0.01, reg_all=0.1)
algo.fit(trainset)

In [141]:
test_pred = algo.test(testset)

In [142]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8567


0.8567204630995651

In [145]:
algo.predict(uid=146.0, iid='Toy Story (1995)').est

3.7152047218438713

Теперь у меня есть 2 обученных алгоритма. Пробую собрать общую гибридную систему 

In [230]:
def recommend_for_user(userid):
#   отбираем фильмы, которым пользоватль поставил масимальные оценки
#   если пользователь оценил много фильмов, оставляем 5 наиболее высоко оцененных
    top_full = dataset[dataset['uid']==1].sort_values(by='rating', ascending=False)['iid'].values
    top_short=[]
    if len(top_full) > 5:
        top_short = top_full[:5]
    else:
        top_short=top_full
    
    similar_films=set()
#   используем 1ю модель  
    for film in top_short:
 
        distances,suggestions=model.kneighbors(movie_pivot.loc[film,:].values.reshape(1,-1))

        for i in range(len(suggestions[0])):
#           если фильм уже просмотрен - пропускаем  
            if movie_pivot.index[suggestions[0][i]] in top_full:
                continue
            similar_films.add(movie_pivot.index[suggestions[0][i]])
    
    similar_films = list(similar_films)
    film_score={}
    
#   используем 2ю модель     
    for sim_film in similar_films:
        est = algo.predict(uid=userid, iid=sim_film).est 
        film_score[sim_film] = est
        
    sorted_films = sorted(film_score.items(), key=lambda x: x[1], reverse=True)
#   возвращаем 10 первых значений (если всего получилось меньше 10, возвращаем сколько есть) 
    if len(sorted_films) > 10:
        sorted_films = sorted_films[:10]
        
    return sorted_films
  

Получим рекоммендации

In [231]:
recommend_for_user(1)

[('Sweet Hereafter, The (1997)', 4.919666964154644),
 ('Manchurian Candidate, The (1962)', 4.745793849294549),
 ('Crimes and Misdemeanors (1989)', 4.698483789829483),
 ('Maltese Falcon, The (1941)', 4.657360642065175),
 ('Butch Cassidy and the Sundance Kid (1969)', 4.6517825202797205),
 ('Killing Fields, The (1984)', 4.6506057372870355),
 ('Breaking Away (1979)', 4.644613350379735),
 ('Player, The (1992)', 4.634065631477329),
 ('Fear and Loathing in Las Vegas (1998)', 4.630739949858216),
 ('Jaws (1975)', 4.615940141320188)]

In [234]:
recommend_for_user(42)

[('For Your Eyes Only (1981)', 4.418362958785652),
 ('Jaws (1975)', 4.297529703857469),
 ('Crimes and Misdemeanors (1989)', 4.289439582946648),
 ('Deliverance (1972)', 4.246352026194097),
 ('Chariots of Fire (1981)', 4.212867447374068),
 ('Breaking Away (1979)', 4.201303244807084),
 ('Maltese Falcon, The (1941)', 4.189064388881887),
 ('Spy Who Loved Me, The (1977)', 4.164587063676464),
 ('Sweet Hereafter, The (1997)', 4.147214676194469),
 ('Doors, The (1991)', 4.142280464037425)]

In [235]:
dataset[dataset['uid']==42].sort_values(by='rating', ascending=False)[:10]

,uid,iid,rating
38596,42.0,Saving Private Ryan (1998),5.0
19461,42.0,"Time to Kill, A (1996)",5.0
23823,42.0,On Golden Pond (1981),5.0
23663,42.0,Top Gun (1986),5.0
23412,42.0,"Doors, The (1991)",5.0
23282,42.0,Platoon (1986),5.0
23157,42.0,Reservoir Dogs (1992),5.0
22621,42.0,Swingers (1996),5.0
46862,42.0,American Pie (1999),5.0
47061,42.0,Eyes Wide Shut (1999),5.0


In [240]:
recommend_for_user(55)

[('Hope and Glory (1987)', 3.8321732177345647),
 ('Crimes and Misdemeanors (1989)', 3.8321550154540183),
 ('Player, The (1992)', 3.7685636987893942),
 ('Terminator 2: Judgment Day (1991)', 3.641559550608656),
 ('Sweet Hereafter, The (1997)', 3.6373682909360885),
 ('Fish Called Wanda, A (1988)', 3.6237452764752205),
 ('Manchurian Candidate, The (1962)', 3.6184685056659074),
 ('Killing Fields, The (1984)', 3.5777518662772114),
 ('Mission, The (1986)', 3.5630803829522137),
 ('Fifth Element, The (1997)', 3.5019298777949714)]

In [242]:
dataset[dataset['uid']==55].sort_values(by='rating', ascending=False)[:10]

,uid,iid,rating
44837,55.0,"Lock, Stock & Two Smoking Barrels (1998)",5.0
83534,55.0,"Departed, The (2006)",5.0
78678,55.0,Layer Cake (2004),5.0
59230,55.0,Snatch (2000),5.0
53235,55.0,"Boondock Saints, The (2000)",5.0
82146,55.0,Lucky Number Slevin (2006),4.5
79711,55.0,Crash (2004),4.5
85546,55.0,"Bourne Ultimatum, The (2007)",4.0
28857,55.0,Highlander (1986),4.0
29486,55.0,Gandhi (1982),4.0
